# STCL Locking Workflow with RedPitayaSTCL

This notebook provides a complete desktop workflow for setting up and operating the **Scanning Transfer Cavity Lock (STCL)** using the `RedPitayaSTCL` codebase.

It is organized as:
1. Clone + install + basic software validation.
2. Verify Red Pitaya OS / SSH / SCPI services.
3. Generate a **triangular ramp on OUT2** and acquire on **IN1**.
4. Run STCL in sequence:
   - Connect and start STCL server.
   - Start cavity scan.
   - Start cavity lock.
   - Start laser lock.

> Update all IP addresses and lock parameters before running on hardware.

## 1) Clone, install, and test the codebase

Run the next cells on your desktop machine (Linux/macOS shell style).  
For Windows, use WSL or adapt commands to PowerShell.

In [ ]:
# --- User-adjustable paths ---
from pathlib import Path

WORK_DIR = Path.home() / "lab"
REPO_URL = "https://github.com/<your-org-or-user>/RedPitayaSTCL.git"  # <- replace if needed
REPO_DIR = WORK_DIR / "RedPitayaSTCL"
VENV_DIR = WORK_DIR / ".venv-stcl"

WORK_DIR.mkdir(parents=True, exist_ok=True)
print(f"Working directory: {WORK_DIR}")

In [ ]:
# Clone (or reuse) repository
import subprocess

if not REPO_DIR.exists():
    subprocess.run(["git", "clone", REPO_URL, str(REPO_DIR)], check=True)
else:
    print(f"Repository already exists: {REPO_DIR}")

subprocess.run(["git", "-C", str(REPO_DIR), "status", "--short"], check=False)

In [ ]:
# Create virtual environment + install dependencies
import subprocess, sys

subprocess.run([sys.executable, "-m", "venv", str(VENV_DIR)], check=True)
python_bin = VENV_DIR / ("Scripts/python.exe" if (VENV_DIR / "Scripts/python.exe").exists() else "bin/python")
pip_bin = VENV_DIR / ("Scripts/pip.exe" if (VENV_DIR / "Scripts/pip.exe").exists() else "bin/pip")

subprocess.run([str(pip_bin), "install", "--upgrade", "pip", "setuptools", "wheel"], check=True)
subprocess.run([str(pip_bin), "install", "paramiko", "numpy", "matplotlib", "scipy", "jupyter"], check=True)

print("Python:", python_bin)

In [ ]:
# Basic import / path test against this repo
import subprocess, textwrap

python_code = textwrap.dedent(f"""
import sys
sys.path.insert(0, r"{REPO_DIR}")

import lockclient
import communication
import redpitaya_scpi

print('Imports OK')
print('lockclient:', lockclient.__file__)
""")

result = subprocess.run([str(python_bin), "-c", python_code], capture_output=True, text=True)
print(result.stdout)
if result.returncode != 0:
    print(result.stderr)
    raise RuntimeError("Import test failed")

## 2) Check Red Pitaya OS, SSH, and SCPI service

This section verifies:
- Reachability (`ping`)
- SSH login and OS info
- SCPI port (`5000`) availability

> Default Red Pitaya credentials are often `root` / `root`.

In [ ]:
# --- Hardware address configuration ---
RP_CAV_IP = "192.168.0.101"   # scanning cavity RP
RP_LOCK1_IP = "192.168.0.102" # first laser lock RP
RP_MON_IP = "192.168.0.100"   # monitor RP

RP_SSH_USER = "root"
RP_SSH_PASS = "root"
SCPI_PORT = 5000

In [ ]:
# Ping + SSH (OS check) + SCPI port check
import socket
import subprocess
import paramiko

def ping_once(ip):
    return subprocess.run(["ping", "-c", "1", "-W", "1", ip], capture_output=True, text=True).returncode == 0

for ip in [RP_CAV_IP, RP_LOCK1_IP, RP_MON_IP]:
    print(f"Ping {ip}:", "OK" if ping_once(ip) else "FAILED")

ssh = paramiko.SSHClient()
ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())
ssh.connect(RP_CAV_IP, username=RP_SSH_USER, password=RP_SSH_PASS, timeout=5)
stdin, stdout, stderr = ssh.exec_command("uname -a && cat /etc/redpitaya_version || cat /opt/redpitaya/version")
print("
=== RP_CAV OS INFO ===")
print(stdout.read().decode().strip())
err = stderr.read().decode().strip()
if err:
    print("STDERR:", err)
ssh.close()

sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
sock.settimeout(2)
try:
    sock.connect((RP_CAV_IP, SCPI_PORT))
    print(f"SCPI TCP port {SCPI_PORT} reachable on {RP_CAV_IP}")
finally:
    sock.close()

## 3) Verify signal path: generate triangular ramp (OUT2) and acquire on IN1

This test confirms end-to-end analog functionality:
- Configure SCPI generator on **OUT2** with triangle waveform.
- Trigger acquisition and read **IN1** samples.
- Plot the captured signal.

### Wiring
- Connect `OUT2` to `IN1` on the same Red Pitaya (directly or via attenuator as needed).
- Ensure input gain/jumper settings match expected voltage range.

In [ ]:
# SCPI waveform generation + acquisition test
import sys
import time
import numpy as np
import matplotlib.pyplot as plt

sys.path.insert(0, str(REPO_DIR))
from redpitaya_scpi import scpi

rp = scpi(RP_CAV_IP)

# Reset generator state and set OUT2 triangular ramp
rp.tx_txt("GEN:RST")
rp.sour_set(chan=2, func="TRIANGLE", volt=0.4, freq=300, offset=0.0)
rp.tx_txt("OUTPUT2:STATE ON")

# Configure acquisition on IN1
rp.acq_set(dec=8, trig_lvl=0.0, units="VOLTS", sample_format="ASCII", averaging=True, gain=["LV","LV"])
rp.tx_txt("ACQ:RST")
rp.tx_txt("ACQ:START")
rp.tx_txt("ACQ:TRIG CH1_PE")

# Wait until trigger event arrives
timeout_s = 2.0
start = time.time()
while time.time() - start < timeout_s:
    status = rp.txrx_txt("ACQ:TRIG:STAT?")
    if status == "TD":
        break
    time.sleep(0.01)
else:
    raise TimeoutError("Acquisition trigger timeout. Check wiring/trigger settings.")

samples = rp.acq_data(chan=1, lat=True, num_samples=4096, convert=True)
rp.tx_txt("OUTPUT2:STATE OFF")
rp.close()

samples = np.array(samples, dtype=float)
print("Acquired samples:", samples.size, "min/max:", samples.min(), samples.max())

plt.figure(figsize=(10, 3))
plt.plot(samples, lw=1)
plt.title("IN1 acquisition while driving OUT2 triangular waveform")
plt.xlabel("Sample index")
plt.ylabel("Voltage [V]")
plt.grid(True, alpha=0.3)
plt.show()

## 4) Connect and start STCL server

The next cells use `LockClient` / `RP_client` from this repository.

### Expected Red Pitaya roles
- `Cav`: cavity scan/master (`mode="scan"`)
- `Lock1`: laser lock RP (`mode="lock"`)
- `Mon`: monitor RP (`mode="monitor"`)

You can add more lock RPs by extending the dictionary.

In [ ]:
import sys
sys.path.insert(0, str(REPO_DIR))

from lockclient import LockClient, RP_client

RPs = {
    "Cav": RP_client((RP_CAV_IP, 5000), {}, mode="scan"),
    "Lock1": RP_client((RP_LOCK1_IP, 5000), {}, mode="lock"),
    "Mon": RP_client((RP_MON_IP, 5000), {}, mode="monitor"),
}

Lock = LockClient(RPs)

# Starts server process on each RP over SSH and then starts client event loop
Lock.connect_all()
Lock.start()
print("STCL communication initialized")

## 5) Start cavity scan

Use this section to:
- set cavity/reference ranges and lockpoint,
- run cavity scanning loop,
- start/stop monitor RP.

In [ ]:
# Parameter updates for cavity peak finding / stabilization
Lock.update_setting("Cav", "Master", "range", [[0.10, 0.40], [1.70, 2.00]])
Lock.update_setting("Cav", "Master", "lockpoint", 1.80)
Lock.update_setting("Cav", "Master", "enabled", True)

# Optional PID tuning for cavity lock loop
Lock.update_setting("Cav", "Master", "PID", {"P": 0.08, "I": 0.001, "D": 0.0, "limit": [-1.0, 1.0]})

print("Cavity parameters sent")

In [ ]:
# Start/stop cavity scan and monitor
Lock.start_scan("Cav")
Lock.start_monitor("Mon")   # opens live cavity monitor window

# Manual actions when needed:
# Lock.stop_monitor("Mon")
# Lock.stop_loop("Cav")

## 6) Start cavity lock

When scan alignment is good, stop free scan loop and start cavity lock.

In [ ]:
# Stop scan loop first, then lock cavity
Lock.stop_loop("Cav")
Lock.start_lock("Cav")
print("Cavity lock command issued")

# To stop cavity lock:
# Lock.stop_loop("Cav")

## 7) Start laser lock

Configure lock channels (`Slave1` / `Slave2`) and enable locking outputs.

In [ ]:
# Example: configure and enable Lock1 -> Slave1 (OUT1)
Lock.update_setting("Lock1", "Slave1", "enabled", True)
Lock.update_setting("Lock1", "Slave1", "range", [0.50, 0.80])
Lock.update_setting("Lock1", "Slave1", "lockpoint", 0.60)
Lock.update_setting("Lock1", "Slave1", "PID", {"P": 0.05, "I": 0.0008, "D": 0.0, "limit": [-1.0, 1.0]})

# Optional second laser channel on same RP
Lock.update_setting("Lock1", "Slave2", "enabled", False)
# Lock.update_setting("Lock1", "Slave2", "range", [1.05, 1.25])
# Lock.update_setting("Lock1", "Slave2", "lockpoint", 1.15)
# Lock.update_setting("Lock1", "Slave2", "PID", {"P": 0.04, "I": 0.0005, "D": 0.0, "limit": [-1.0, 1.0]})

# Start / stop laser lock loop on RP
Lock.start_lock("Lock1")
print("Laser lock started")

# To stop laser lock:
# Lock.stop_loop("Lock1")

## 8) Monitoring and safe shutdown

Use error monitoring for lightweight long-duration tracking, then shut everything down cleanly.

In [ ]:
# Optional error monitor
# Lock.stop_monitor("Mon")
# Lock.start_error_monitor("Mon", tmin=30e-3)

# Stop individual loops manually if needed:
# Lock.stop_loop("Lock1")
# Lock.stop_loop("Cav")

# Full cleanup (recommended at end of session)
# Lock.close()

---
### Notes
- If any RP becomes unresponsive, power-cycle it and rerun the connection cells.
- Keep cavity scan and lock ranges conservative first, then tighten after stable operation.
- Save tuned JSON settings from `settings/*.json` for reproducible startup.